In [2]:
from hive_rc_auto.helpers.rc_delegation import RCAllData, RCListOfAccounts, get_mongo_db, get_utc_now_timestamp
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from hive_rc_auto.helpers.config import Config


ImportError: dlopen(/Users/bol/Library/Caches/pypoetry/virtualenvs/hive-rc-auto-BnIDKwNA-py3.11/lib/python3.11/site-packages/pymssql/_mssql.cpython-311-darwin.so, 0x0002): symbol not found in flat namespace '_bcp_batch'

In [ ]:

db_name = (
    "rc_history_testnet" if Config.TESTNET else "rc_history"
)
db_rc_history = get_mongo_db(db_name)

: 

In [ ]:
cursor = db_rc_history.find({"real_mana":{"$ne":None}}, {"_id": 0})

data = []
async for doc in cursor:
    data.append(doc)


: 

In [ ]:

df = pd.DataFrame(data)
df.set_index("timestamp", inplace=True)


df["age"] = datetime.utcnow() - df.index
df["age_hours"] = df.age.dt.total_seconds()/3600
all_accounts = df.account.unique()
all_accounts.sort()

: 

In [ ]:
df[df.delegating =="delegating"]

: 

In [ ]:

cursor = db_rc_history.find({"real_mana":None}, {"_id": 0})

data = []
async for doc in cursor:
    data.append(doc)
    
df_rc_changes = pd.DataFrame(data)
df_rc_changes["age"] = datetime.utcnow() - df_rc_changes.timestamp
df_rc_changes.set_index('timestamp', inplace=True)
df_rc_changes["age_hours"] = df_rc_changes.age.dt.total_seconds()/3600

: 

In [ ]:
hive_acc = "podping.aaa"
df_rc_change = df_rc_changes[df_rc_changes.account == hive_acc]
for index, row in df_rc_change.iterrows():
    print( row.delegated_rc)

: 

In [ ]:
hive_acc = "podping.aaa"
dfa = df[df.account == hive_acc]
dfa.resample('10T').mean(numeric_only=True)

: 

In [ ]:
for hive_acc in all_accounts:
    dfa = df[df.account == hive_acc]
    # dfa.resample('30T').mean(numeric_only=True)
    df_rc_change = df_rc_changes[df_rc_changes.account == hive_acc]
    
    if dfa.real_mana_percent.iloc[-1] < 95:      
          
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(
            go.Scatter(x=dfa.age_hours, y=dfa['real_mana_percent'],name="RC %"),
            secondary_y=False,
        )            
        fig.add_trace(
            go.Scatter(x=dfa.age_hours, y=dfa['real_mana'], name="RC"),
            secondary_y=True,
        ) 
        
        # fig.add_trace(
        #     go.Scatter(x=df_rc_change.index, y=df_rc_change.delegated_rc, name="Changes"),
        #     secondary_y=True
        # )
        
        # for index, row in df_rc_change.iterrows():
        #     up_down = "down" if row.cut else "up"
        #     fig.add_vline(x=row.age_hours, line_width=0.6, line_dash="dash", line_color="green")

           
        # Set x-axis title
        fig.update_xaxes(title_text=f"Age - {hive_acc}")

        fig.update_layout(
            title_text=f"<b>{hive_acc}</b> Resource Credits"
        )        
        fig.update_layout(title_x=0.2, title_y=0.2)
        
        # Set y-axes titles
        fig.update_yaxes(title_text="<b>RC %</b>", secondary_y=False)
        fig.update_yaxes(title_text="<b>RC</b>", secondary_y=True)
        yaxes_range_max = dfa['real_mana_percent'].max() + 5
        fig.update_yaxes(range=[0, yaxes_range_max], secondary_y=False)
        fig.update_layout(showlegend=True,)
        fig.update_layout(legend_x=0.80, legend_y=0.85)
        fig.update_xaxes(autorange="reversed")
        fig.update_layout(margin={"autoexpand": True, "b": 0, "t":0, "l":0, "r":0})
        fig.show()


: 